# Week 10: Trajectory Optimization

### Topics Covered

- Cost Functions (smoothness, speed, safety); Model Predictive Control (MPC) for planning and control integration; Real-time constraints

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand the key concepts
2. Implement algorithms
3. Apply techniques to real-world problems

---

## Setup

Import required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, Polygon, FancyArrowPatch
from matplotlib.animation import FuncAnimation
from scipy.optimize import minimize
from scipy.interpolate import CubicSpline

# Set random seed for reproducibility
np.random.seed(42)

## 1. Trajectory Optimization Fundamentals

Trajectory optimization finds the **best** trajectory that satisfies constraints while minimizing a cost function.

### Trajectory vs. Path

- **Path**: Geometric curve in space (no time information)
  - Example: $(x(s), y(s))$ where $s \in [0, 1]$
  
- **Trajectory**: Path with timing information
  - Example: $(x(t), y(t), v(t))$ where $t$ is time

### Optimization Problem Formulation

**Decision Variables:** Trajectory parameters $\mathbf{z} = [x_0, x_1, \ldots, x_N, u_0, u_1, \ldots, u_{N-1}]$

**Minimize:**
$$J(\mathbf{z}) = \phi(x_N) + \sum_{k=0}^{N-1} L(x_k, u_k)$$

**Subject to:**
- **Dynamics:** $x_{k+1} = f(x_k, u_k)$
- **Initial conditions:** $x_0 = x_{init}$
- **Goal constraints:** $x_N \in \mathcal{X}_{goal}$
- **State constraints:** $x_k \in \mathcal{X}_{safe}$
- **Control constraints:** $u_k \in \mathcal{U}$

Where:
- $x_k$: State at time step $k$ (position, velocity, heading)
- $u_k$: Control input (steering, acceleration)
- $\phi(x_N)$: Terminal cost
- $L(x_k, u_k)$: Stage cost

### Cost Function Components

**1. Smoothness:**
$$J_{smooth} = \int_0^T \left( \left(\frac{d^2x}{dt^2}\right)^2 + \left(\frac{d^2y}{dt^2}\right)^2 \right) dt$$
Penalizes high accelerations (jerk minimization)

**2. Speed:**
$$J_{speed} = \int_0^T (v_{ref} - v(t))^2 dt$$
Tracks desired speed

**3. Safety (Obstacle Avoidance):**
$$J_{safety} = \sum_{obstacles} \max(0, d_{safe} - d(x, obs))^2$$
Penalizes being too close to obstacles

**4. Control Effort:**
$$J_{control} = \int_0^T (u(t)^T R u(t)) dt$$
Minimizes steering and acceleration inputs

**Total Cost:**
$$J = w_1 J_{smooth} + w_2 J_{speed} + w_3 J_{safety} + w_4 J_{control}$$

In [ ]:
# Simple MPC Implementation for Vehicle Tracking

class SimpleMPC:
    """Model Predictive Controller for 2D point mass"""
    
    def __init__(self, horizon=10, dt=0.1, Q=None, R=None):
        """
        Initialize MPC
        
        Parameters:
        - horizon: Prediction horizon steps
        - dt: Time step
        - Q: State cost matrix (4x4)
        - R: Control cost matrix (2x2)
        """
        self.N = horizon
        self.dt = dt
        
        # Cost matrices
        self.Q = Q if Q is not None else np.diag([10.0, 10.0, 1.0, 1.0])  # [x, y, vx, vy]
        self.R = R if R is not None else np.diag([0.1, 0.1])  # [ax, ay]
        
        # Control limits
        self.u_max = np.array([3.0, 3.0])  # Max acceleration (m/s²)
        self.u_min = -self.u_max
    
    def dynamics(self, state, control):
        """
        Simple 2D point mass dynamics
        state: [x, y, vx, vy]
        control: [ax, ay]
        """
        x, y, vx, vy = state
        ax, ay = control
        
        # Integrate with Euler method
        x_next = x + vx * self.dt
        y_next = y + vy * self.dt
        vx_next = vx + ax * self.dt
        vy_next = vy + ay * self.dt
        
        return np.array([x_next, y_next, vx_next, vy_next])
    
    def cost(self, z, x0, x_ref):
        """
        Compute total cost for decision variables z
        
        z: Flattened array of [u_0, ..., u_{N-1}]
        x0: Initial state
        x_ref: Reference trajectory (N+1 x 4)
        """
        # Reshape controls
        controls = z.reshape((self.N, 2))
        
        # Simulate forward
        states = [x0]
        total_cost = 0.0
        
        for k in range(self.N):
            # State cost
            x_error = states[k] - x_ref[k]
            total_cost += x_error @ self.Q @ x_error
            
            # Control cost
            u = controls[k]
            total_cost += u @ self.R @ u
            
            # Propagate dynamics
            x_next = self.dynamics(states[k], u)
            states.append(x_next)
        
        # Terminal cost
        x_error_final = states[-1] - x_ref[-1]
        total_cost += x_error_final @ (self.Q * 10) @ x_error_final
        
        return total_cost
    
    def solve(self, x0, x_ref):
        """
        Solve MPC optimization
        
        x0: Current state
        x_ref: Reference trajectory (N+1 x 4)
        
        Returns: Optimal control sequence
        """
        # Initial guess (zero acceleration)
        u_init = np.zeros(self.N * 2)
        
        # Bounds on controls
        bounds = [(self.u_min[i % 2], self.u_max[i % 2]) for i in range(self.N * 2)]
        
        # Solve optimization
        result = minimize(
            fun=lambda z: self.cost(z, x0, x_ref),
            x0=u_init,
            method='SLSQP',
            bounds=bounds,
            options={'maxiter': 100, 'disp': False}
        )
        
        # Reshape to controls
        u_opt = result.x.reshape((self.N, 2))
        return u_opt


# Simulate MPC tracking a reference trajectory
def simulate_mpc_tracking():
    # Reference trajectory (figure-8)
    t_ref = np.linspace(0, 10, 100)
    x_ref = 20 * np.sin(t_ref)
    y_ref = 10 * np.sin(2 * t_ref)
    
    # Reference velocities (numerical derivative)
    vx_ref = np.gradient(x_ref, t_ref)
    vy_ref = np.gradient(y_ref, t_ref)
    
    reference_traj = np.column_stack([x_ref, y_ref, vx_ref, vy_ref])
    
    # Initialize MPC
    mpc = SimpleMPC(horizon=10, dt=0.1)
    
    # Initial state
    x_current = np.array([0.0, 0.0, 0.0, 0.0])
    
    # Simulate
    states = [x_current]
    controls = []
    
    num_steps = len(reference_traj) - mpc.N - 1
    
    print("Running MPC simulation...")
    for i in range(num_steps):
        # Get reference for horizon
        x_ref_horizon = reference_traj[i:i+mpc.N+1]
        
        # Solve MPC
        u_opt = mpc.solve(x_current, x_ref_horizon)
        
        # Apply first control
        u_apply = u_opt[0]
        controls.append(u_apply)
        
        # Simulate system
        x_current = mpc.dynamics(x_current, u_apply)
        states.append(x_current)
        
        if i % 10 == 0:
            print(f"  Step {i}/{num_steps}")
    
    states = np.array(states)
    controls = np.array(controls)
    
    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Trajectory tracking
    ax1 = axes[0, 0]
    ax1.plot(reference_traj[:, 0], reference_traj[:, 1], 'g--', linewidth=2, label='Reference', alpha=0.7)
    ax1.plot(states[:, 0], states[:, 1], 'b-', linewidth=2, label='MPC Tracking')
    ax1.plot(states[0, 0], states[0, 1], 'go', markersize=10, label='Start')
    ax1.set_xlabel('X Position (m)')
    ax1.set_ylabel('Y Position (m)')
    ax1.set_title('MPC Trajectory Tracking')
    ax1.legend()
    ax1.grid(True)
    ax1.axis('equal')
    
    # Tracking error
    ax2 = axes[0, 1]
    tracking_error = np.linalg.norm(states[:num_steps, :2] - reference_traj[:num_steps, :2], axis=1)
    time_vec = np.arange(num_steps) * mpc.dt
    ax2.plot(time_vec, tracking_error, 'r-', linewidth=2)
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Position Error (m)')
    ax2.set_title(f'Tracking Error (Mean: {np.mean(tracking_error):.2f}m)')
    ax2.grid(True)
    
    # Control inputs
    ax3 = axes[1, 0]
    ax3.plot(time_vec, controls[:, 0], label='ax (longitudinal)')
    ax3.plot(time_vec, controls[:, 1], label='ay (lateral)')
    ax3.axhline(mpc.u_max[0], color='r', linestyle='--', alpha=0.5, label='Limits')
    ax3.axhline(mpc.u_min[0], color='r', linestyle='--', alpha=0.5)
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Acceleration (m/s²)')
    ax3.set_title('Control Inputs')
    ax3.legend()
    ax3.grid(True)
    
    # Velocity profile
    ax4 = axes[1, 1]
    speed = np.linalg.norm(states[:, 2:4], axis=1)
    ref_speed = np.linalg.norm(reference_traj[:len(speed), 2:4], axis=1)
    ax4.plot(time_vec, speed[:num_steps], 'b-', label='Actual Speed', linewidth=2)
    ax4.plot(time_vec, ref_speed[:num_steps], 'g--', label='Reference Speed', linewidth=2, alpha=0.7)
    ax4.set_xlabel('Time (s)')
    ax4.set_ylabel('Speed (m/s)')
    ax4.set_title('Velocity Tracking')
    ax4.legend()
    ax4.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nMPC Performance:")
    print(f"  Mean tracking error: {np.mean(tracking_error):.3f} m")
    print(f"  Max tracking error: {np.max(tracking_error):.3f} m")
    print(f"  Mean control magnitude: {np.mean(np.linalg.norm(controls, axis=1)):.3f} m/s²")

simulate_mpc_tracking()

---

## Exercises

### Exercise 1: Multi-Objective Cost Function Tuning

**Objective:** Implement and tune a trajectory optimizer with multiple competing objectives.

**Task:** Create a trajectory optimization problem that balances:
1. Minimizing travel time
2. Maximizing comfort (minimizing acceleration)
3. Maintaining safe distance from obstacles

**Instructions:**
- Define cost weights for each objective
- Implement obstacle avoidance cost using distance penalties
- Visualize the Pareto front showing trade-offs between objectives
- Compare trajectories with different weight combinations

### Exercise 2: MPC with Vehicle Dynamics

**Objective:** Extend the MPC implementation to use realistic vehicle dynamics instead of point mass.

**Task:** Implement MPC with the bicycle kinematic model:

$$
\begin{align}
\dot{x} &= v \cos(\theta) \\
\dot{y} &= v \sin(\theta) \\
\dot{\theta} &= \frac{v}{L} \tan(\delta) \\
\dot{v} &= a
\end{align}
$$

Where:
- $(x, y)$ = position
- $\theta$ = heading angle
- $v$ = velocity
- $\delta$ = steering angle (control input)
- $a$ = acceleration (control input)
- $L$ = wheelbase length

**Instructions:**
- Modify the `SimpleMPC` class to use bicycle model dynamics
- Add constraints on steering angle: $|\delta| \leq \delta_{max}$
- Add constraints on steering rate: $|\dot{\delta}| \leq \dot{\delta}_{max}$
- Test on a lane change maneuver

### Exercise 3: Receding Horizon Obstacle Avoidance

**Objective:** Implement MPC-based real-time obstacle avoidance in a dynamic environment.

**Task:** Create an MPC controller that:
1. Tracks a reference path
2. Avoids moving obstacles within the prediction horizon
3. Re-plans at each time step as obstacles move

**Instructions:**
- Add obstacle positions as time-varying constraints
- Implement collision checking within the optimization cost
- Use soft constraints (penalty method) for obstacle avoidance
- Visualize the predicted trajectory and actual path in real-time
- Compare performance with different prediction horizons (N = 5, 10, 20)

In [ ]:
# Exercise Solutions

# Exercise 1: Multi-Objective Cost Function Tuning
# TODO: Implement multi-objective trajectory optimization
# 
# Suggested approach:
# 1. Define separate cost functions for each objective
# 2. Create weighted sum: J_total = w1*J_time + w2*J_comfort + w3*J_safety
# 3. Vary weights and collect optimal solutions
# 4. Plot Pareto front showing trade-offs
#
# Example structure:
# def multi_objective_cost(traj, weights, obstacles):
#     J_time = trajectory_duration(traj)
#     J_comfort = integrate_squared_acceleration(traj)
#     J_safety = sum([penalty(distance(traj, obs)) for obs in obstacles])
#     return weights[0]*J_time + weights[1]*J_comfort + weights[2]*J_safety


# Exercise 2: MPC with Vehicle Dynamics
# TODO: Implement bicycle model in MPC
#
# Suggested approach:
# 1. Create BicycleMPC class extending SimpleMPC
# 2. Redefine dynamics() method with bicycle kinematic model
# 3. Update state vector to [x, y, theta, v]
# 4. Update control vector to [delta, a]
# 5. Add steering constraints in bounds
#
# Example structure:
# class BicycleMPC(SimpleMPC):
#     def __init__(self, wheelbase=2.7, **kwargs):
#         super().__init__(**kwargs)
#         self.L = wheelbase
#         self.delta_max = np.deg2rad(30)  # Max steering angle
#         
#     def dynamics(self, state, control):
#         x, y, theta, v = state
#         delta, a = control
#         
#         # Bicycle model (use small-angle approximation or exact)
#         x_next = x + v * np.cos(theta) * self.dt
#         y_next = y + v * np.sin(theta) * self.dt
#         theta_next = theta + (v / self.L) * np.tan(delta) * self.dt
#         v_next = v + a * self.dt
#         
#         return np.array([x_next, y_next, theta_next, v_next])


# Exercise 3: Receding Horizon Obstacle Avoidance
# TODO: Implement MPC with dynamic obstacle avoidance
#
# Suggested approach:
# 1. Extend SimpleMPC to include obstacle cost in cost function
# 2. Predict obstacle positions over horizon
# 3. Add soft constraint penalty for collision
# 4. Re-solve at each time step
#
# Example structure:
# class ObstacleAvoidanceMPC(SimpleMPC):
#     def __init__(self, obstacle_radius=1.0, safety_margin=0.5, **kwargs):
#         super().__init__(**kwargs)
#         self.r_obs = obstacle_radius
#         self.r_safe = safety_margin
#     
#     def obstacle_cost(self, state, obstacles_t):
#         """Compute collision penalty for given state and obstacle positions"""
#         x, y = state[0], state[1]
#         penalty = 0.0
#         
#         for obs in obstacles_t:
#             dist = np.sqrt((x - obs[0])**2 + (y - obs[1])**2)
#             min_dist = self.r_obs + self.r_safe
#             
#             if dist < min_dist:
#                 # Quadratic penalty inside safety zone
#                 penalty += 1000 * (min_dist - dist)**2
#         
#         return penalty
#     
#     def cost(self, z, x0, x_ref, obstacles_trajectory):
#         """Modified cost including obstacle avoidance"""
#         controls = z.reshape((self.N, 2))
#         states = [x0]
#         total_cost = 0.0
#         
#         for k in range(self.N):
#             # Tracking cost
#             x_error = states[k] - x_ref[k]
#             total_cost += x_error @ self.Q @ x_error
#             
#             # Control cost
#             u = controls[k]
#             total_cost += u @ self.R @ u
#             
#             # Obstacle avoidance cost
#             total_cost += self.obstacle_cost(states[k], obstacles_trajectory[k])
#             
#             # Dynamics
#             x_next = self.dynamics(states[k], u)
#             states.append(x_next)
#         
#         # Terminal cost
#         x_error_final = states[-1] - x_ref[-1]
#         total_cost += x_error_final @ (self.Q * 10) @ x_error_final
#         
#         return total_cost


---

## References

### Books

1. **Betts, J. T.** (2010). *Practical Methods for Optimal Control and Estimation Using Nonlinear Programming* (2nd ed.). SIAM.
   - Comprehensive treatment of trajectory optimization methods
   - Chapter 4: Direct transcription methods
   - Chapter 7: Real-time optimization

2. **Boyd, S., & Vandenberghe, L.** (2004). *Convex Optimization*. Cambridge University Press.
   - Foundation for optimization theory
   - Available free online: https://web.stanford.edu/~boyd/cvxbook/

3. **Rawlings, J. B., Mayne, D. Q., & Diehl, M.** (2017). *Model Predictive Control: Theory, Computation, and Design* (2nd ed.). Nob Hill Publishing.
   - Definitive reference for MPC
   - Chapter 1: Introduction to MPC concepts
   - Chapter 2: MPC stability and constraint handling

4. **Lavalle, S. M.** (2006). *Planning Algorithms*. Cambridge University Press.
   - Chapter 14: Optimal motion planning
   - Free online: http://planning.cs.uiuc.edu/

### Papers

5. **Coulter, R. C.** (1992). *Implementation of the Pure Pursuit Path Tracking Algorithm*. CMU-RI-TR-92-01.
   - Classic paper on trajectory tracking
   - Foundation for many autonomous vehicle controllers

6. **Werling, M., et al.** (2010). "Optimal trajectory generation for dynamic street scenarios in a Frenet Frame." *IEEE International Conference on Robotics and Automation (ICRA)*.
   - Frenet frame approach to trajectory optimization
   - Widely used in industry for highway driving

7. **Ziegler, J., & Stiller, C.** (2009). "Spatiotemporal state lattices for fast trajectory planning in dynamic on-road driving scenarios." *IEEE/RSJ International Conference on Intelligent Robots and Systems*.
   - State lattice planning with temporal dimension
   - Used in DARPA Urban Challenge winner

8. **Paden, B., et al.** (2016). "A Survey of Motion Planning and Control Techniques for Self-Driving Urban Vehicles." *IEEE Transactions on Intelligent Vehicles*, 1(1), 33-55.
   - Excellent overview of planning and control methods
   - Section IV covers trajectory optimization

9. **Verschueren, R., et al.** (2019). "Towards a modular software package for embedded optimization." *IFAC-PapersOnLine*, 52(1), 164-169.
   - Modern computational methods for real-time MPC
   - Code generation for embedded systems

### Online Resources

10. **MathWorks - Understanding Model Predictive Control**
    - https://www.mathworks.com/videos/series/understanding-model-predictive-control.html
    - Video series explaining MPC fundamentals

11. **CasADi - Optimization Framework**
    - https://web.casadi.org/
    - Powerful symbolic framework for numerical optimization
    - Widely used for trajectory optimization in research

12. **ACADO Toolkit**
    - http://acado.github.io/
    - Open-source toolkit for automatic control and dynamic optimization
    - Implements multiple direct methods

13. **CVXPy - Convex Optimization**
    - https://www.cvxpy.org/
    - Python library for convex optimization
    - Example: https://www.cvxpy.org/examples/applications/optimal_control.html

### Software & Tools

14. **Drake** - Model-based design and verification for robotics
    - https://drake.mit.edu/
    - Trajectory optimization examples: https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/%2Ftrajectory_optimization.ipynb

15. **OMPL** - Open Motion Planning Library
    - https://ompl.kavrakilab.org/
    - Includes optimization-based planners

16. **PyDy** - Python Dynamics
    - https://www.pydy.org/
    - Symbolic dynamics for complex mechanical systems

### Research Groups & Courses

17. **Stanford AA203: Optimal and Learning-Based Control**
    - Course materials: https://stanfordasl.github.io/aa203/
    - Lecture notes on trajectory optimization and MPC

18. **MIT 6.832: Underactuated Robotics**
    - http://underactuated.mit.edu/
    - Chapter 10: Trajectory optimization
    - Interactive notebooks with examples

19. **Berkeley MPC Lab**
    - https://borrelli.me.berkeley.edu/
    - Research on MPC for autonomous vehicles
    - Publications and code repositories

### Related Topics

20. **Kelly, M.** (2017). "An Introduction to Trajectory Optimization: How to Do Your Own Direct Collocation." *SIAM Review*, 59(4), 849-904.
    - Tutorial paper with MATLAB code
    - Excellent practical introduction to direct methods

21. **Schulman, J., et al.** (2013). "Finding Locally Optimal, Collision-Free Trajectories with Sequential Convex Optimization." *Robotics: Science and Systems*.
    - TrajOpt algorithm for robot motion planning
    - Handles non-convex constraints efficiently